In [1]:
import pandas as pd
import os
import shutil

In [2]:
basedir = '/data/CARD/PD/GP2/raw_genotypes'
out_genotypes = '/data/CARD/PD/GP2/genotypes'
shulman_ny_path = f'{basedir}/shulman_ny'
gtc_file_path = f'{shulman_ny_path}/GP2_GCT_files'
idat_file_path = f'{shulman_ny_path}/GP2_Shulman'
key_file = f'{gtc_file_path}/Key File_FINAL_Shulman_and_NY_011421.txt'
manifest_txt_path = f'{gtc_file_path}/FINALSS_after_rerun__Shulman_and_NY_011421.csv'
bpm = f'{gtc_file_path}/NeuroBooster_20042459_A1.bpm'
cluster_file = f'{gtc_file_path}/NBSCluster_file_n1393_011921.egt'
pheno_file = f'{gtc_file_path}/BCM_20210201_samples.csv'

#software paths
GTCtoVCF = 'GTCtoVCF/gtc_to_vcf.py'
iaap = '../executables/iaap-cli-linux-x64-1.1.0-sha.80d7e5b3d9c1fdfc2e99b472a90652fd3848bbc7/iaap-cli/iaap-cli'

ref_fasta = 'ref_data/hg38_ref.fa'
shulman_gtc_path = f'{basedir}/SHULMAN/gtc_files'
shulman_idat_path = f'{basedir}/SHULMAN/idats'
shulman_out = f'{out_genotypes}/SHULMAN'
ny_gtc_path = f'{basedir}/NY/gtc_files'
ny_idat_path = f'{basedir}/NY/idats'
ny_out = f'{out_genotypes}/NY'

In [3]:
# create updated ID and phenotype files for later
manifest = pd.read_csv(manifest_txt_path, header=10)
pheno = pd.read_csv(pheno_file)
pheno = pheno.astype({'Original_clinicalID': str})
manifest = manifest.astype({'Sample_ID': str})
pheno_out = manifest.merge(pheno, how='left', left_on='Sample_ID', right_on='Original_clinicalID')
pheno_out['IID'] = pheno_out.SentrixBarcode_A.astype(str) + '_' + pheno_out.SentrixPosition_A.astype(str)
pheno_out['FID'] = 0
pheno_out['FID_new'] = 0
pheno_out['pheno'] = 0
pheno_out.loc[pheno_out.Phenotype == 'PD', 'pheno'] = 2
pheno_out.loc[pheno_out.Phenotype == 'Control', 'pheno'] = 1
shulman_pheno = pheno_out[pheno_out.Study=='Shulman']
ny_pheno = pheno_out[pheno_out.Study=='NY']
shulman_pheno[['FID','IID', 'FID_new', 'Sample_ID']].to_csv(f'{shulman_ny_path}/shulman_update_ids.txt', sep='\t', header=False, index=False)
shulman_pheno[['FID_new', 'Sample_ID', 'pheno']].to_csv(f'{shulman_ny_path}/shulman_update_pheno.txt', sep='\t', header=False, index=False)
ny_pheno[['FID','IID', 'FID_new', 'Sample_ID']].to_csv(f'{shulman_ny_path}/ny_update_ids.txt', sep='\t', header=False, index=False)
ny_pheno[['FID_new', 'Sample_ID', 'pheno']].to_csv(f'{shulman_ny_path}/ny_update_pheno.txt', sep='\t', header=False, index=False)


In [ ]:
# create new directories to store split cohorts
!mkdir {basedir}/SHULMAN
!mkdir {basedir}/NY
!mkdir {basedir}/SHULMAN/idats
!mkdir {basedir}/NY/idats
!mkdir {out_genotypes}
!mkdir {out_genotypes}/SHULMAN
!mkdir {out_genotypes}/SHULMAN/ped
!mkdir {out_genotypes}/NY
!mkdir {out_genotypes}/NY/ped
!mkdir {shulman_out}/plink
!mkdir {shulman_out}/plink/indiv_samples
!mkdir {ny_out}/plink

In [7]:
shulman_pheno.to_csv(f'{out_genotypes}/SHULMAN/shulman_covariates.txt', sep='\t', index=False, header=True)
ny_pheno.to_csv(f'{out_genotypes}/NY/ny_covariates.txt', sep='\t', index=False, header=True)

In [15]:
# create filenaames and split manifest into respective cohorts
manifest['filename'] = manifest['SentrixBarcode_A'].astype(str) + '_' + manifest['SentrixPosition_A']
shulman = manifest.loc[manifest.Study == 'Shulman']
ny = manifest.loc[manifest.Study == 'NY']

In [ ]:
# create directory for each sentrix barcode A if they don't already exist-- 
def split_cohort_idats(manifest_df, name, idat_file_path):
        
    for code in manifest_df.SentrixBarcode_A.unique():
        if os.path.exists(f'{basedir}/{name}/idats/{code}'):
            print(f'{basedir}/{name}/idats/{code} already exists')
        else:
            os.mkdir(f'{basedir}/{name}/idats/{code}')

    missing_idats = []

    # split idats into respective cohorts and populated each sentrix barcode A directory
    for i, filename in enumerate(manifest_df.filename):
        sentrix_code = manifest_df.SentrixBarcode_A.iloc[i]
        grn = f'{idat_file_path}/{sentrix_code}/{filename}_Grn.idat'
        red = f'{idat_file_path}/{sentrix_code}/{filename}_Red.idat'

        if os.path.isfile(grn):
            shutil.copyfile(src=grn, dst=f'{basedir}/{name}/idats/{sentrix_code}/{filename}_Grn.idat')
        else:
            missing_idats.append(grn)
            
        if os.path.isfile(red):
            shutil.copyfile(src=red, dst=f'{basedir}/{name}/idats/{sentrix_code}/{filename}_Red.idat')
        else:
            missing_idats.append(red)
        

    len(missing_idats)

In [ ]:
split_cohort_idats(manifest_df=shulman, name='SHULMAN', idat_file_path=idat_file_path)
split_cohort_idats(manifest_df=ny, name='NY', idat_file_path=idat_file_path)


In [ ]:
# swarm command to run in parallel
with open('shulman_idat_to_ped.swarm','w') as f:
    
    for code in shulman.SentrixBarcode_A.unique():
        
        shulman_idat_to_ped_cmd = f'\
{iaap} gencall \
{bpm} \
{cluster_file} \
{shulman_out}/ped/ \
-f {shulman_idat_path}/{code} \
-p \
-t 8'
        
        f.write(f'{shulman_idat_to_ped_cmd}\n')
f.close()

In [ ]:
# !swarm -f shulman_idat_to_ped.swarm -g 32 -t 16 --time=10:00:00 --logdir swarm --gres=lscratch:20 --partition=norm

In [ ]:
# copy map file to match name of each ped
map = f'{shulman_out}/ped/NeuroBooster_20042459_A1.map'
for filename in shulman.filename:
    ped = f'{shulman_out}/ped/{filename}.ped'
    out_map = f'{shulman_out}/ped/{filename}.map'
    if os.path.isfile(ped):
        shutil.copyfile(src=map, dst=out_map)
    else:
        print(f'{ped} does not exist!')
        print(f'{out_map} creation cancelled')


In [ ]:
for filename in shulman.filename:
    ped = f'{shulman_out}/ped/{filename}'
    make_bed_cmd = f'\
plink \
--file {ped} \
--make-bed \
--out {shulman_out}/plink/indiv_samples/{filename}'

    shell_do(make_bed_cmd)

In [16]:
# write plink merge command
with open("shulman_merge_bed.list", 'w') as f:
    for filename in shulman.filename:
        bed = f'{shulman_out}/plink/indiv_samples/{filename}'
        f.write(f'{bed}\n')
f.close()

with open("shulman_merge.swarm", 'w') as f:

    plink_merge_cmd = f'\
plink \
--merge-list shulman_merge_bed.list \
--update-ids {shulman_ny_path}/shulman_update_ids.txt \
--make-bed \
--out {shulman_out}/plink/shulman'
    f.write(f"{plink_merge_cmd}")
f.close()

In [20]:
# !swarm -f shulman_merge.swarm -g 64 -t 32 --time=10:00:00 --logdir swarm --gres=lscratch:20 --partition=norm
!{plink_merge_cmd}

PLINK v1.90b4.4 64-bit (21 May 2017)           www.cog-genomics.org/plink/1.9/
(C) 2005-2017 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /data/CARD/PD/GP2/genotypes/SHULMAN/plink/shulman.log.
Options in effect:
  --make-bed
  --merge-list shulman_merge_bed.list
  --out /data/CARD/PD/GP2/genotypes/SHULMAN/plink/shulman
  --update-ids /data/CARD/PD/GP2/raw_genotypes/shulman_ny/shulman_update_ids.txt

1031832 MB RAM detected; reserving 515916 MB for main workspace.
same position.
108786 more same-position warnings: see log file.
Performing single-pass merge (1288 people, 2004347 variants).
Merged fileset written to
/data/CARD/PD/GP2/genotypes/SHULMAN/plink/shulman-merge.bed +
/data/CARD/PD/GP2/genotypes/SHULMAN/plink/shulman-merge.bim +
/data/CARD/PD/GP2/genotypes/SHULMAN/plink/shulman-merge.fam .
2004347 variants loaded from .bim file.
1288 people (707 males, 558 females, 23 ambiguous) loaded from .fam.
Ambiguous sex IDs written to
/data/CARD/PD/GP2/genoty

In [22]:
!plink --bfile {shulman_out}/plink/shulman_merge --pheno {shulman_ny_path}/shulman_update_pheno.txt --make-bed --out {shulman_out}/plink/shulman

PLINK v1.90b4.4 64-bit (21 May 2017)           www.cog-genomics.org/plink/1.9/
(C) 2005-2017 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /data/CARD/PD/GP2/genotypes/SHULMAN/plink/shulman.log.
Options in effect:
  --bfile /data/CARD/PD/GP2/genotypes/SHULMAN/plink/shulman_merge
  --make-bed
  --out /data/CARD/PD/GP2/genotypes/SHULMAN/plink/shulman
  --pheno /data/CARD/PD/GP2/raw_genotypes/shulman_ny/shulman_update_pheno.txt

1031832 MB RAM detected; reserving 515916 MB for main workspace.
2004347 variants loaded from .bim file.
1288 people (707 males, 558 females, 23 ambiguous) loaded from .fam.
Ambiguous sex IDs written to
/data/CARD/PD/GP2/genotypes/SHULMAN/plink/shulman.nosex .
809 phenotype values present after --pheno.
phenotypes to be ignored, use the --allow-no-sex flag.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1288 founders and 0 nonfounders present.
Calculating allele frequencies... 101112131415161718192

In [ ]:
# get basic statistics
plink_miss_cmd = f'\
plink \
--bfile {shulman_out}/plink/shulman \
--missing \
--out {shulman_out}/plink/shulman'

shell_do(plink_miss_cmd)

In [ ]:
# get average call rate
lmiss = pd.read_csv(f'{shulman_out}/plink/shulman.lmiss', sep='\s+')
imiss = pd.read_csv(f'{shulman_out}/plink/shulman.imiss', sep='\s+')
avg_call_rate = 100-lmiss.F_MISS.mean()
avg_geno_rate = 100-imiss.F_MISS.mean()
print(f'Average Call Rate: {avg_call_rate}')
print(f'Average Genotyping Rate: {avg_geno_rate}')

In [18]:
!cat shulman_merge_bed.list


    

0028_R07C01
/data/CARD/PD/GP2/genotypes/SHULMAN/plink/indiv_samples/204697840028_R08C01
/data/CARD/PD/GP2/genotypes/SHULMAN/plink/indiv_samples/204701860027_R01C01
/data/CARD/PD/GP2/genotypes/SHULMAN/plink/indiv_samples/204701860027_R02C01
/data/CARD/PD/GP2/genotypes/SHULMAN/plink/indiv_samples/204701860027_R03C01
/data/CARD/PD/GP2/genotypes/SHULMAN/plink/indiv_samples/204701860027_R04C01
/data/CARD/PD/GP2/genotypes/SHULMAN/plink/indiv_samples/204701860027_R05C01
/data/CARD/PD/GP2/genotypes/SHULMAN/plink/indiv_samples/204701860027_R06C01
/data/CARD/PD/GP2/genotypes/SHULMAN/plink/indiv_samples/204701860027_R07C01
/data/CARD/PD/GP2/genotypes/SHULMAN/plink/indiv_samples/204701860027_R08C01
/data/CARD/PD/GP2/genotypes/SHULMAN/plink/indiv_samples/204701870051_R01C01
/data/CARD/PD/GP2/genotypes/SHULMAN/plink/indiv_samples/204701870051_R02C01
/data/CARD/PD/GP2/genotypes/SHULMAN/plink/indiv_samples/204701870051_R03C01
/data/CARD/PD/GP2/genotypes/SHULMAN/plink/indiv_samples/204701870051_R05C01


In [22]:
f'{shulman_ny_path}/ny_update_ids.txt'

In [2]:
shulman_ny_path

NameError: name 'shulman_ny_path' is not defined

In [24]:
pheno_out[['FID','IID']]

,FID,IID
0,0,204697840024_R01C01
1,0,204697840024_R02C01
2,0,204697840024_R03C01
3,0,204697840024_R04C01
4,0,204697840024_R05C01
...,...,...
1388,0,204701870002_R05C01
1389,0,204701870002_R06C01
1390,0,204701870002_R07C01
1391,0,204701870002_R08C01


In [31]:
shulman

'/data/CARD/PD/GP2/genotypes/SHULMAN/plink/shulman'